In [1]:
import pandas as pd
import numpy as np
import random
import os
import tensorflow as tf
import sys
sys.path.append('..')
from modules import utils, constants 
from modules.env import LupusEnv

/home/lmuyama/anaconda3/envs/stable_baselines_tf2_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/lmuyama/anaconda3/envs/stable_baselines_tf2_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/lmuyama/anaconda3/envs/stable_baselines_tf2_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.i

In [2]:
SEED = 126
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
tf.set_random_seed(SEED)
tf.compat.v1.set_random_seed(SEED)
SEED

126

In [3]:
test_data = pd.read_csv('../data/test_set_constant.csv')
test_data = test_data.fillna(-1)
test_data.head()

,ana,fever,leukopenia,thrombocytopenia,auto_immune_hemolysis,delirium,psychosis,seizure,non_scarring_alopecia,oral_ulcers,...,joint_involvement,proteinuria,anti_cardioliphin_antibodies,anti_β2gp1_antibodies,lupus_anti_coagulant,low_c3,low_c4,anti_dsdna_antibody,anti_smith_antibody,label
0,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
1,1,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,1
2,1,0,0,0,0,0,1,0,0,0,...,1,0,0,0,1,1,0,0,0,1
3,1,1,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,1,0,1
4,1,0,0,0,0,0,0,0,1,0,...,1,0,0,0,0,1,0,0,0,1


In [4]:
X_test = test_data.iloc[:, 0:-1]
y_test = test_data.iloc[:, -1]
X_test, y_test = np.array(X_test), np.array(y_test)
X_test.shape, y_test.shape

((14000, 23), (14000,))

In [5]:
model_name = 'dueling_dqn_per'
steps = int(10e7)

In [6]:
dqn_model = utils.load_dqn(f'../models/logs/{model_name}/risk_factor/seed_{SEED}_{steps}/best_acc_model.zip')
test_df = utils.evaluate_dqn(dqn_model, X_test, y_test)
acc, f1, roc_auc = utils.test(test_df.y_actual, test_df.y_pred)
print(f'{model_name}: {acc}, {f1}, {roc_auc}, {test_df.episode_length.min()}, {test_df.episode_length.mean()}, {test_df.episode_length.max()}')

Loading a model without an environment, this model cannot be trained until it has a valid environment.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use tf.cast instead.
The environment seed is [126]
Testing done.....
dueling_dqn_per: 97.74285714285715, 97.73607975107359, 97.67995521211564, 6.0, 14.938357142857143, 24.0


In [7]:
test_df.to_csv(f'../test_dfs/{model_name}_risk_factor_{SEED}_{steps}.csv', index=False)